# 1. SEP28 PREPARATION

In [1]:
!unzip sep28_all_together

unzip:  cannot find or open sep28_all_together, sep28_all_together.zip or sep28_all_together.ZIP.


In [34]:
!ls ./sep28_all_together/StutteringIsCool*

ls: cannot access './sep28_all_together/StutteringIsCool*': No such file or directory


In [2]:
import os

root_folder = './sep28_all_together/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 21856


In [3]:
import pandas as pd

df_sep = pd.read_csv('SEP_28k_labels.csv')
df_sep.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0


In [ ]:
print(df_sep[(df_sep['Show'] == 'WomenWhoStutter') & (df_sep['EpId'] == 99) & (df_sep['ClipId'] == 99)]['NoStutteredWords'])

27324    2
Name: NoStutteredWords, dtype: int64


In [6]:
tmp = df_sep[(df_sep['NoStutteredWords'] == 3) 
                        & (df_sep['Unsure'] == 0)
                        & (df_sep['PoorAudioQuality'] == 0)
                        & (df_sep['Prolongation'] == 0)
                        & (df_sep['Block'] == 0)
                        & (df_sep['SoundRep'] == 0)    
                        & (df_sep['WordRep'] == 0)
                        & (df_sep['DifficultToUnderstand'] == 0)
                        & (df_sep['Interjection'] == 0)
                        & (df_sep['NaturalPause'] == 0)
                        & (df_sep['Music'] == 0)
                        & (df_sep['NoSpeech'] == 0)]

len(df_sep), len(tmp)

(28177, 2479)

In [77]:
col_list_nostutter = [ 
            'Unsure', 
            'PoorAudioQuality',
            'Prolongation', 
            'Block', 
            'SoundRep',
            'WordRep',
            'DifficultToUnderstand',
            'Interjection',
            # 'NaturalPause',
            # 'Music',
            # 'NoSpeech'
            ]

tmp = df_sep[(df_sep['NoStutteredWords'] == 3) & (df_sep['Show'] == 'WomenWhoStutter') & (df_sep[col_list_nostutter].sum(axis=1) == 0)]

len(df_sep), len(tmp), tmp.head()

(28177,
 1283,
                   Show  EpId  ClipId     Start      Stop  Unsure  \
 19055  WomenWhoStutter     0      41  30782080  30830080       0   
 19074  WomenWhoStutter     0      60  33892640  33940640       0   
 19083  WomenWhoStutter     0      69  37276640  37324640       0   
 19088  WomenWhoStutter     0      74  37539040  37587040       0   
 19109  WomenWhoStutter     0      95  39892160  39940160       0   
 
        PoorAudioQuality  Prolongation  Block  SoundRep  WordRep  \
 19055                 0             0      0         0        0   
 19074                 0             0      0         0        0   
 19083                 0             0      0         0        0   
 19088                 0             0      0         0        0   
 19109                 0             0      0         0        0   
 
        DifficultToUnderstand  Interjection  NoStutteredWords  NaturalPause  \
 19055                      0             0                 3             0   
 

### Creating no stutter files with 'NoStutteredWords', 'NaturalPause', 'Music', 'NoSpeech'  and count of 3458 

In [41]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './sep28_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './sep28_balanced_7'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if lst[i] != 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [8, 9, 10, 11]

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if check_elements(features, specific_positions):
        print(show_name,pod_id,clip_id, features,df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)][['NoStutteredWords','NaturalPause','Music','NoSpeech']])
        count += 1
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)

print(count)

StutterTalk 33 151 [0 0 0 0 0 0 0 0 3 0 0 0]        NoStutteredWords  NaturalPause  Music  NoSpeech
12052                 3             0      0         0
WomenWhoStutter 6 90 [0 0 0 0 0 0 0 0 3 1 0 0]        NoStutteredWords  NaturalPause  Music  NoSpeech
19915                 3             1      0         0
StutterTalk 63 94 [0 0 0 0 0 0 0 0 3 0 0 0]        NoStutteredWords  NaturalPause  Music  NoSpeech
13693                 3             0      0         0
WomenWhoStutter 50 68 [0 0 0 0 0 0 0 0 3 1 0 0]        NoStutteredWords  NaturalPause  Music  NoSpeech
23262                 3             1      0         0
StutterTalk 70 8 [0 0 0 0 0 0 0 0 3 0 0 0]        NoStutteredWords  NaturalPause  Music  NoSpeech
14368                 3             0      0         0
WomenWhoStutter 42 28 [0 0 0 0 0 0 0 0 3 0 0 0]        NoStutteredWords  NaturalPause  Music  NoSpeech
22662                 3             0      0         0
WomenWhoStutter 57 181 [0 0 0 0 0 0 0 0 3 1 0 0]        NoStutter

In [71]:
import os

root_folder = './sep28_balanced_7/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 3458


### Creating stutter files for  'Prolongation', 'Block', 'SoundRep', 'WordRep', 'Interjection' to be detected as stuttering situation and count of 3324

In [60]:
# tmp = df_sep[(df_sep['NoStutteredWords'] == 0) 
#                         & (df_sep['Unsure'] == 0)
#                         & (df_sep['PoorAudioQuality'] == 0)
#                         # & (df_sep['Prolongation'] == 0)
#                         & (df_sep['Block'] == 0)
#                         & ((df_sep['SoundRep'] >= 2) | (df_sep['Interjection'] >= 2) | ((df_sep['Prolongation'] == 0))    
#                         # & (df_sep['WordRep'] >= 2)
#                         & (df_sep['DifficultToUnderstand'] == 0)
#                         # & (df_sep['Interjection'] == 0)
#                         & (df_sep['NaturalPause'] == 0)
#                         & (df_sep['Music'] == 0)
#                         & (df_sep['NoSpeech'] == 0)]

tmp = df_sep[(df_sep['NoStutteredWords'] == 0) 
                        & (df_sep['Unsure'] == 0)
                        & (df_sep['PoorAudioQuality'] == 0)
                        & (df_sep['Prolongation'] != 0)
                        & (df_sep['Block'] != 0)
                        & ((df_sep['SoundRep'] != 0))    
                        & (df_sep['WordRep'] != 0)
                        & (df_sep['DifficultToUnderstand'] == 0)
                        & (df_sep['Interjection'] != 0)
                        & (df_sep['NaturalPause'] == 0)
                        & (df_sep['Music'] == 0)
                        & (df_sep['NoSpeech'] == 0)]

len(df_sep), len(tmp)

(28177, 46)

In [78]:
col_list_stutter = [ 
            # 'Unsure', 
            # 'PoorAudioQuality',
            'Prolongation', 
            'Block', 
            'SoundRep',
            'WordRep',
            # 'DifficultToUnderstand',
            'Interjection',
            # 'NaturalPause',
            # 'Music',
            # 'NoSpeech'
            ]

col_list_nostutter = [ 
            'Unsure',
            'NoStutteredWords', 
            'PoorAudioQuality',
            # 'Prolongation', 
            # 'Block', 
            # 'SoundRep',
            # 'WordRep',
            'DifficultToUnderstand',
            # 'Interjection',
            'NaturalPause',
            'Music',
            'NoSpeech'
            ]

tmp = df_sep[(df_sep['Show'] == 'WomenWhoStutter') & (df_sep[col_list_nostutter].sum(axis=1) == 0) & (df_sep[col_list_stutter].sum(axis=1) != 0)]

len(df_sep), len(tmp)

(28177, 1520)

In [72]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './sep28_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './sep28_balanced_7'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if lst[i] != 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [2, 3, 4, 5, 7]

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if check_elements(features, specific_positions):
        print(show_name,pod_id,clip_id)
        print(features,df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)][['Prolongation', 'Block','SoundRep','WordRep','Interjection']])
        count += 1

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
    if count == 3458:
        break


print(count)

[0 0 1 2 0 0 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep  Interjection
24160             1      2         0        0             0
WomenWhoStutter 87 121
[0 0 0 1 0 3 0 2 0 0 0 0]        Prolongation  Block  SoundRep  WordRep  Interjection
26197             0      1         0        3             2
WomenWhoStutter 88 163
[0 0 0 1 2 2 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep  Interjection
26445             0      1         2        2             0
HeStutters 4 19
[0 0 3 2 0 0 0 0 0 0 0 0]      Prolongation  Block  SoundRep  WordRep  Interjection
588             3      2         0        0             0
HeStutters 1 249
[0 0 2 1 1 0 0 1 0 0 0 0]      Prolongation  Block  SoundRep  WordRep  Interjection
289             2      1         1        0             1
WomenWhoStutter 97 131
[0 0 0 0 3 0 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep  Interjection
27115             0      0         3        0             0
StutterTalk 39 33
[0 0 0 1 2 2

In [73]:
import os

root_folder = './sep28_balanced_7/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)


Total number of files: 6782


### Renaming files in sep28_balanced to nostutter (3458) and stutter files(3324):

In [74]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './sep28_balanced_7/'

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]

    
    if features[8] != 0:
        print(features)
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "nostutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

    else:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "stutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 1 0 0]
[0 0 0 0 0 0 0 0 3 2 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 1 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 2 0 0]
[0 0 0 0 0 0 0 0 3 2 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 2 0 0]
[0 0 0 0 0 0 0 0 3 2 0 0]
[0 0 0 0 0 0 0 0 3 2 0 0]
[0 0 0 0 0 0 0 0 3 1 0 0]
[0 0 0 0 0 0 0 0 3 1 0 0]
[0 0 0 0 0 0 0 0 3 1 0 0]
[0 0 0 0 0 0 0 0 3 0 3 2]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 3 0 0]
[0 0 0 0 0 0 0 0 3 0 3 1]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 2 0 0]
[0 0 0 0 0 0 0 0 3 1 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 2 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 3 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 2 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 2 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0

In [75]:
import os

# Specify the directory where your .wav files are located
directory = './sep28_balanced_7/'

# Initialize a counter variable
count = 0
count1 = 0
# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.startswith('nostutter'):
        count += 1
    if filename.startswith('stutter'):
        count1 += 1

print(f"Number of nostutter and stutter files: {count} , {count1} and tot {count+count1}")

Number of nostutter and stutter files: 3458 , 3324 and tot 6782


In [79]:
!ls ./sep28_balanced_7/

nostutter_WomenWhoStutter_15_33.wav    stutter_WomenWhoStutter_34_149.wav
nostutter_WomenWhoStutter_15_3.wav     stutter_WomenWhoStutter_34_155.wav
nostutter_WomenWhoStutter_1_54.wav     stutter_WomenWhoStutter_34_156.wav
nostutter_WomenWhoStutter_15_6.wav     stutter_WomenWhoStutter_34_157.wav
nostutter_WomenWhoStutter_16_13.wav    stutter_WomenWhoStutter_34_189.wav
nostutter_WomenWhoStutter_16_16.wav    stutter_WomenWhoStutter_34_18.wav
nostutter_WomenWhoStutter_16_21.wav    stutter_WomenWhoStutter_34_190.wav
nostutter_WomenWhoStutter_1_62.wav     stutter_WomenWhoStutter_34_194.wav
nostutter_WomenWhoStutter_16_35.wav    stutter_WomenWhoStutter_34_197.wav
nostutter_WomenWhoStutter_1_67.wav     stutter_WomenWhoStutter_34_202.wav
nostutter_WomenWhoStutter_1_70.wav     stutter_WomenWhoStutter_34_207.wav
nostutter_WomenWhoStutter_17_12.wav    stutter_WomenWhoStutter_34_209.wav
nostutter_WomenWhoStutter_17_22.wav    stutter_WomenWhoStutter_34_21.wav
nostutter_WomenWhoStutter_17_25.wav    s

### Splitting to train, val, test sets

In [76]:
import random
import shutil
import os

# Specify the source directory where your files are located
source_directory = './sep28_balanced_7'

# Specify the destination directories for the split files
destination_directory_1 = './train_sep28_7'
destination_directory_2 = './val_sep28_7'
destination_directory_3 = './test_sep28_7'

# Create the destination directories if they don't exist
for directory in [destination_directory_1, destination_directory_2, destination_directory_3]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Shuffle the files in the source directory
files = os.listdir(source_directory)
random.shuffle(files)

# Define the sizes for each split
split_sizes = [0.8, 0.1, 0.1]

# Calculate the number of files for each split
total_files = len(files)
split_counts = [int(size * total_files) for size in split_sizes]

# Split the files into different lists based on the split counts
file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

count = 0
# Move the files to the destination directories
for i, file_split in enumerate(file_splits):
    if i == 0:
        destination_directory = destination_directory_1
    elif i == 1:
        destination_directory = destination_directory_2
    else:
        destination_directory = destination_directory_3

    for filename in file_split:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
        count += 1

print(count)


6781


In [80]:
# Specify the directory where your .wav files are located
directory1 = './train_sep28_7'
directory2 = './val_sep28_7'
directory3 = './test_sep28_7'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    if filename.startswith('nostutter'):
        count_train += 1
    if filename.endswith('.wav'):
        count_train_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    if filename.startswith('nostutter'):
        count_val += 1
    if filename.endswith('.wav'):
        count_val_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    if filename.startswith('nostutter'):
        count_test += 1    
    if filename.endswith('.wav'):
        count_test_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

print('**********Sep28 dataset**********')
print(f"Number of nostutter files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of nostutter files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of nostutter files in the test folder: {count_test} and tot is {count_test_tot}")

**********Sep28 dataset**********
Number of nostutter files in the train folder: 2771 and tot is 5425
Number of nostutter files in the val folder: 344 and tot is 678
Number of nostutter files in the test folder: 343 and tot is 678


In [ ]:
# import random
# import shutil
# import os
# import tensorflow as tf

# # Specify the source directory where your files are located
# source_directory = './fluency_bank_all_together'

# # Specify the destination directories for the split files
# destination_directory_1 = './fluency_balanced'
# destination_directory_2 = './fluency_inference'

# # Create the destination directories if they don't exist
# for directory in [destination_directory_1, destination_directory_2]:
#     if not os.path.exists(directory):
#         os.makedirs(directory)

# # Shuffle the files in the source directory
# files = os.listdir(source_directory)
# random.shuffle(files)

# # Calculate the number of files for each split
# total_files = len(files)
# split_counts = [800, total_files-800]

# # Split the files into different lists based on the split counts
# file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]]]


# def check_elements(lst, positions):
#     non_zero_count = 0

#     for i in range(len(lst)):
#         if i in positions:
#             if i == 0 and lst[i] == 3:
#                 non_zero_count += 1
#             elif i != 0 and lst[i] >= 0:
#                 non_zero_count += 1
#         elif lst[i] != 0:
#             return False
#     return  non_zero_count >= 1

# specific_positions = [8, 9, 10, 11]

# count = 0
# # Move the files to the destination directories
# for i, file_split in enumerate(file_splits):
#     if i == 0:
#         destination_directory = destination_directory_1
#     else:
#         destination_directory = destination_directory_2

#     for filename in file_split:

#         path_parts = tf.strings.split(filename, '/')
#         path_end = path_parts[-1]
#         file_parts = tf.strings.split(path_end, '_')
#         show_name = file_parts[0].numpy().decode('utf-8')
#         pod_id = int(file_parts[-2])
#         clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
#         features = df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]

#         if check_elements(features, specific_positions):
#             print(show_name,pod_id,clip_id)
#             print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['NoStutteredWords','NaturalPause','Music','NoSpeech']])
                


#             # source_path = os.path.join(source_directory, filename)
#             # destination_path = os.path.join(destination_directory, filename)
#             # shutil.copy2(source_path, destination_path)
    
#             count += 1

# print(count)

# 2. FLUENCY BANK PREPARATION

In [ ]:
!unzip ./fluency_bank_all_together.zip

  inflating: fluency_bank/FluencyBank_173_53.wav  
  inflating: fluency_bank/FluencyBank_173_54.wav  
  inflating: fluency_bank/FluencyBank_173_55.wav  
  inflating: fluency_bank/FluencyBank_173_56.wav  
  inflating: fluency_bank/FluencyBank_173_57.wav  
  inflating: fluency_bank/FluencyBank_173_58.wav  
  inflating: fluency_bank/FluencyBank_173_59.wav  
  inflating: fluency_bank/FluencyBank_173_6.wav  
  inflating: fluency_bank/FluencyBank_173_60.wav  
  inflating: fluency_bank/FluencyBank_173_61.wav  
  inflating: fluency_bank/FluencyBank_173_62.wav  
  inflating: fluency_bank/FluencyBank_173_63.wav  
  inflating: fluency_bank/FluencyBank_173_64.wav  
  inflating: fluency_bank/FluencyBank_173_65.wav  
  inflating: fluency_bank/FluencyBank_173_66.wav  
  inflating: fluency_bank/FluencyBank_173_67.wav  
  inflating: fluency_bank/FluencyBank_173_68.wav  
  inflating: fluency_bank/FluencyBank_173_69.wav  
  inflating: fluency_bank/FluencyBank_173_7.wav  
  inflating: fluency_bank/Fluency

In [81]:
!ls ./fluency_bank_all_together

FluencyBank_024_111.wav  FluencyBank_096_84.wav   FluencyBank_173_56.wav
FluencyBank_024_112.wav  FluencyBank_096_85.wav   FluencyBank_173_57.wav
FluencyBank_024_113.wav  FluencyBank_096_86.wav   FluencyBank_173_58.wav
FluencyBank_024_114.wav  FluencyBank_096_87.wav   FluencyBank_173_59.wav
FluencyBank_024_115.wav  FluencyBank_096_88.wav   FluencyBank_173_5.wav
FluencyBank_024_116.wav  FluencyBank_096_89.wav   FluencyBank_173_60.wav
FluencyBank_024_117.wav  FluencyBank_096_8.wav	  FluencyBank_173_61.wav
FluencyBank_024_118.wav  FluencyBank_096_90.wav   FluencyBank_173_62.wav
FluencyBank_024_119.wav  FluencyBank_096_91.wav   FluencyBank_173_63.wav
FluencyBank_024_11.wav	 FluencyBank_096_92.wav   FluencyBank_173_64.wav
FluencyBank_024_120.wav  FluencyBank_096_93.wav   FluencyBank_173_65.wav
FluencyBank_024_121.wav  FluencyBank_096_94.wav   FluencyBank_173_66.wav
FluencyBank_024_122.wav  FluencyBank_096_9.wav	  FluencyBank_173_67.wav
FluencyBank_024_123.wav  FluencyBank_097_0.wav	  Fluenc

In [ ]:
import os

root_folder = './fluency_bank_all_together/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 4144


In [82]:
import pandas as pd

df_flu = pd.read_csv('fluencybank_labels.csv')
df_flu.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,FluencyBank,10,0,88960,136960,0,0,0,1,0,0,0,0,2,0,0,0
1,FluencyBank,10,1,1271520,1319520,0,0,0,0,0,0,0,0,3,0,0,0
2,FluencyBank,10,2,1813760,1861760,0,0,1,0,0,0,0,0,2,0,0,0
3,FluencyBank,10,3,1842720,1890720,0,0,1,0,0,0,0,0,2,1,0,0
4,FluencyBank,10,4,1893280,1941280,0,0,0,0,0,3,0,0,0,0,0,0


In [ ]:
column_location = df_flu.columns.get_loc('NoStutteredWords')
loc_NoStutteredWords = df_flu.columns.get_loc('NoStutteredWords')
loc_NaturalPause = df_flu.columns.get_loc('NaturalPause')
loc_Music = df_flu.columns.get_loc('Music')
loc_NoSpeech = df_flu.columns.get_loc('NoSpeech')

loc_NoStutteredWords,loc_NaturalPause,loc_Music,loc_NoSpeech

(13, 14, 15, 16)

In [ ]:
listttt = df_flu[(df_flu['Show'] == 'FluencyBank') & (df_flu['EpId'] == 10) & (df_flu['ClipId'] == 0)].to_numpy()[0]
ffff = [0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 0]

loc_NoStutteredWords = df_flu.columns.get_loc('NoStutteredWords')
loc_NaturalPause = df_flu.columns.get_loc('NaturalPause')
loc_Music = df_flu.columns.get_loc('Music')
loc_NoSpeech = df_flu.columns.get_loc('NoSpeech')

def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if i == 0 and lst[i] == 3:
                non_zero_count += 1
            elif i != 0 and lst[i] >= 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [8, 9, 10, 11]

check_elements(ffff, specific_positions), listttt[5:]

(True, array([0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0], dtype=object))

### Creating no stutter files with count of 931 including 'NoStutteredWords', 'NoSpeech',  'Music' and 'NaturalPause'

In [8]:
tmp = df_flu[(df_flu['NoStutteredWords'] == 0) 
                        & (df_flu['Unsure'] == 0)
                        & (df_flu['PoorAudioQuality'] == 0)
                        & (df_flu['Prolongation'] == 0)
                        & (df_flu['Block'] == 0)
                        & (df_flu['SoundRep'] == 0)    
                        & (df_flu['WordRep'] >0)
                        & (df_flu['DifficultToUnderstand'] == 0)
                        & (df_flu['Interjection'] == 0)
                        & (df_flu['NaturalPause'] == 0)
                        & (df_flu['Music'] == 0)
                        & (df_flu['NoSpeech'] == 0)]

len(df_flu), len(tmp)

(4144, 38)

In [83]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './fluency_bank_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './fluency_balanced_7'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if i == 0 and lst[i] == 3:
                non_zero_count += 1
            elif i != 0 and lst[i] >= 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [8, 9, 10, 11]

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features =  df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if check_elements(features, specific_positions):
        print(show_name,pod_id,clip_id)
        print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['NoStutteredWords','NaturalPause','Music','NoSpeech']])
        count += 1

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)


print(count)

[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
3917                 3             0      0         0
FluencyBank 93 25
[0 0 0 0 0 0 0 0 3 1 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
1615                 3             1      0         0
FluencyBank 46 65
[0 0 0 0 0 0 0 0 3 0 0 0]      NoStutteredWords  NaturalPause  Music  NoSpeech
714                 3             0      0         0
FluencyBank 93 7
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
1597                 3             0      0         0
FluencyBank 178 157
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
3332                 3             0      0         0
FluencyBank 219 98
[0 0 0 0 0 0 0 0 3 2 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
3742                 3             2      0         0
FluencyBank 230 92
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
3862   

In [84]:
import os

root_folder = './fluency_balanced_7/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 931


### Creating stutter files for  'Prolongation', 'SoundRep', 'WordRep', 'Block',  'Interjection' to be detected as stuttering situation and count of 923

In [ ]:
tmp = df_flu[(df_flu['NoStutteredWords'] == 0) 
                        & (df_flu['Unsure'] == 0)
                        & (df_flu['PoorAudioQuality'] == 0)
                        & (df_flu['Prolongation'] >= 0)
                        & (df_flu['Block'] >= 0)
                        & (df_flu['SoundRep'] >= 0)    
                        & (df_flu['WordRep'] >= 0)
                        & (df_flu['DifficultToUnderstand'] == 0)
                        & (df_flu['Interjection'] >= 0)
                        & (df_flu['NaturalPause'] == 0)
                        & (df_flu['Music'] == 0)
                        & (df_flu['NoSpeech'] == 0)]

len(df_flu), len(tmp)

(4144, 923)

In [ ]:
loc_Prolongation = df_flu.columns.get_loc('Prolongation')
loc_Block = df_flu.columns.get_loc('Block')
loc_SoundRep = df_flu.columns.get_loc('SoundRep')
loc_WordRep = df_flu.columns.get_loc('WordRep')
loc_Interjection = df_flu.columns.get_loc('Interjection')

loc_Prolongation, loc_Block, loc_SoundRep, loc_WordRep, loc_Interjection

(7, 8, 9, 10, 12)

In [85]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './fluency_bank_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './fluency_balanced_7'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if lst[i] != 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [2, 3, 4, 5, 7]

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features =  df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if check_elements(features, specific_positions):
        print(show_name,pod_id,clip_id)
        print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['Prolongation', 'Block','SoundRep','WordRep','Interjection']])
        count += 1

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)


print(count)

[0 0 1 2 1 0 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep  Interjection
2368             1      2         1        0             0
FluencyBank 132 11
[0 0 1 2 2 1 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep  Interjection
2899             1      2         2        1             0
FluencyBank 121 59
[0 0 0 3 0 0 0 1 0 0 0 0]       Prolongation  Block  SoundRep  WordRep  Interjection
2804             0      3         0        0             1
FluencyBank 99 41
[0 0 1 0 2 0 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep  Interjection
2140             1      0         2        0             0
FluencyBank 132 45
[0 0 2 0 2 2 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep  Interjection
2933             2      0         2        2             0
FluencyBank 81 149
[0 0 0 0 0 3 0 1 0 0 0 0]       Prolongation  Block  SoundRep  WordRep  Interjection
1317             0      0         0        3             1
FluencyBank 109 1
[0 0 0 1 1 3 0 2 0 0 0 0

In [86]:
import os

root_folder = './fluency_balanced_7/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 1854


In [11]:
import os
import random
import shutil

random.seed(32) 

# Set the source folder path
folder_path = './fluency_balanced_7/'

# Get a list of all .wav files in the folder
file_list = [file for file in os.listdir(folder_path) if file.endswith('.wav')]

# Shuffle the file list randomly
random.shuffle(file_list)

# Select the first 100 files
selected_files = file_list[:200]

# Create a destination folder to store the selected files
destination_folder = './fluencyBank_7_200'
os.makedirs(destination_folder, exist_ok=True)

# Move the selected files to the destination folder
for file in selected_files:
    file_path = os.path.join(folder_path, file)
    destination_path = os.path.join(destination_folder, file)
    shutil.copy2(file_path, destination_path)

In [2]:
!ls './fluencyBank_7_200/'

nostutter_FluencyBank_010_16.wav   nostutter_FluencyBank_230_118.wav
nostutter_FluencyBank_010_23.wav   nostutter_FluencyBank_230_11.wav
nostutter_FluencyBank_010_25.wav   nostutter_FluencyBank_230_176.wav
nostutter_FluencyBank_010_34.wav   nostutter_FluencyBank_230_54.wav
nostutter_FluencyBank_010_36.wav   nostutter_FluencyBank_985_14.wav
nostutter_FluencyBank_010_48.wav   nostutter_FluencyBank_985_156.wav
nostutter_FluencyBank_010_57.wav   nostutter_FluencyBank_985_58.wav
nostutter_FluencyBank_010_58.wav   stutter_FluencyBank_016_67.wav
nostutter_FluencyBank_010_5.wav    stutter_FluencyBank_019_41.wav
nostutter_FluencyBank_010_64.wav   stutter_FluencyBank_019_57.wav
nostutter_FluencyBank_010_65.wav   stutter_FluencyBank_024_130.wav
nostutter_FluencyBank_010_95.wav   stutter_FluencyBank_024_194.wav
nostutter_FluencyBank_016_50.wav   stutter_FluencyBank_024_70.wav
nostutter_FluencyBank_016_79.wav   stutter_FluencyBank_046_106.wav
nostutter_FluencyBank_017_32.wav   stutter_FluencyBank_0

In [12]:
!ls ./fluencyBank_7_series/

nostutter_FluencyBank_010_13.wav   nostutter_FluencyBank_230_28.wav
nostutter_FluencyBank_010_14.wav   nostutter_FluencyBank_230_31.wav
nostutter_FluencyBank_010_16.wav   nostutter_FluencyBank_230_33.wav
nostutter_FluencyBank_010_1.wav    nostutter_FluencyBank_230_34.wav
nostutter_FluencyBank_010_21.wav   nostutter_FluencyBank_230_38.wav
nostutter_FluencyBank_010_22.wav   nostutter_FluencyBank_230_41.wav
nostutter_FluencyBank_010_23.wav   nostutter_FluencyBank_230_48.wav
nostutter_FluencyBank_010_25.wav   nostutter_FluencyBank_230_51.wav
nostutter_FluencyBank_010_26.wav   nostutter_FluencyBank_230_54.wav
nostutter_FluencyBank_010_27.wav   nostutter_FluencyBank_230_55.wav
nostutter_FluencyBank_010_30.wav   nostutter_FluencyBank_230_57.wav
nostutter_FluencyBank_010_31.wav   nostutter_FluencyBank_230_59.wav
nostutter_FluencyBank_010_32.wav   nostutter_FluencyBank_230_5.wav
nostutter_FluencyBank_010_34.wav   nostutter_FluencyBank_230_60.wav
nostutter_FluencyBank_010_35.wav   nostutter_Flue

In [2]:
!ls './fluency_bank_all_together/'

FluencyBank_010_0.wav	 FluencyBank_087_38.wav   FluencyBank_121_117.wav
FluencyBank_010_10.wav	 FluencyBank_087_39.wav   FluencyBank_121_118.wav
FluencyBank_010_11.wav	 FluencyBank_087_3.wav	  FluencyBank_121_119.wav
FluencyBank_010_12.wav	 FluencyBank_087_40.wav   FluencyBank_121_11.wav
FluencyBank_010_13.wav	 FluencyBank_087_41.wav   FluencyBank_121_120.wav
FluencyBank_010_14.wav	 FluencyBank_087_42.wav   FluencyBank_121_121.wav
FluencyBank_010_15.wav	 FluencyBank_087_43.wav   FluencyBank_121_122.wav
FluencyBank_010_16.wav	 FluencyBank_087_44.wav   FluencyBank_121_123.wav
FluencyBank_010_17.wav	 FluencyBank_087_45.wav   FluencyBank_121_124.wav
FluencyBank_010_18.wav	 FluencyBank_087_46.wav   FluencyBank_121_125.wav
FluencyBank_010_19.wav	 FluencyBank_087_47.wav   FluencyBank_121_126.wav
FluencyBank_010_1.wav	 FluencyBank_087_48.wav   FluencyBank_121_127.wav
FluencyBank_010_20.wav	 FluencyBank_087_49.wav   FluencyBank_121_128.wav
FluencyBank_010_21.wav	 FluencyBank_087_4.wav	  Fluency

In [4]:
import wave
import audioop

# List of input file names (assuming they are in the same directory as the script)
input_files = ['./fluency_bank_all_together/FluencyBank_046_60.wav',
               './fluency_bank_all_together/FluencyBank_046_61.wav',
               './fluency_bank_all_together/FluencyBank_046_62.wav',
               './fluency_bank_all_together/FluencyBank_046_63.wav',
               './fluency_bank_all_together/FluencyBank_046_64.wav',
               './fluency_bank_all_together/FluencyBank_046_65.wav',
               './fluency_bank_all_together/FluencyBank_046_66.wav',
               './fluency_bank_all_together/FluencyBank_046_67.wav',
               './fluency_bank_all_together/FluencyBank_046_68.wav',
               './fluency_bank_all_together/FluencyBank_046_69.wav',
               './fluency_bank_all_together/FluencyBank_046_70.wav']

# Open the first input file to retrieve the audio parameters
with wave.open(input_files[0], 'rb') as first_file:
    params = first_file.getparams()

    # Create a new output file with the desired parameters
    with wave.open('output.wav', 'wb') as output_file:
        output_file.setparams(params)

        # Loop through the input files and append their audio data to the output file
        for file_name in input_files:
            with wave.open(file_name, 'rb') as input_file:
                # Read the audio data from the input file
                audio_data = input_file.readframes(input_file.getnframes())

                # Write the audio data to the output file
                output_file.writeframes(audio_data)

                # Calculate the remaining time to fill up to 30 seconds
                remaining_frames = params.framerate * (30 - len(input_files) * params.duration)

                # If there is still remaining time, pad it with silence
                if remaining_frames > 0:
                    # Generate silent frames
                    silence_data = audioop.mul(b'\x00' * params.sampwidth, params.sampwidth, 0.0)

                    # Write the silent frames to the output file
                    output_file.writeframes(silence_data * remaining_frames)


AttributeError: '_wave_params' object has no attribute 'duration'

### Renaming files in fluency_balanced to nostutter (931) and stutter files(923):

In [87]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './fluency_balanced_7/'


def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if i == 0 and lst[i] == 3:
                non_zero_count += 1
            elif i != 0 and lst[i] >= 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [8, 9, 10, 11]

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]

    
    if check_elements(features, specific_positions):
        print('Nooooooooooooo',features)
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "nostutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)
        

    else:
        print('YYYYeeeeesssss',features)
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "stutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

Nooooooooooooo [0 0 0 0 0 0 0 0 3 1 0 0]
YYYYeeeeesssssss [0 0 0 2 2 0 0 0 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 1 0 0]
YYYYeeeeesssssss [0 0 1 1 1 0 0 0 0 0 0 0]
YYYYeeeeesssssss [0 0 0 2 1 1 0 0 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 2 0 0]
YYYYeeeeesssssss [0 0 0 1 1 3 0 1 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 1 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 2 0 0]
YYYYeeeeesssssss [0 0 0 0 3 1 0 0 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
YYYYeeeeesssssss [0 0 0 0 3 0 0 0 0 0 0 0]
YYYYeeeeesssssss [0 0 3 0 0 0 0 0 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 2 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
YYYYeeeeesssssss [0 0 0 1 3 0 0 3 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 0 0 0]
YYYYeeeeesssssss [0 0 3 1 0 0 0 0 0 0 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 1 0 0]
Nooooooooooooo [0 0 0 0 0 0 0 0 3 2 0 0

In [89]:
import os

# Specify the directory where your .wav files are located
directory = './fluency_balanced_7/'

# Initialize a counter variable
count = 0
count1 = 0
# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.startswith('nostutter'):
        count += 1
    if filename.startswith('stutter'):
        count1 += 1

print(f"Number of nostutter and stutter files: {count} , {count1} and tot {count+count1}")

Number of nostutter and stutter files: 931 , 923 and tot 1854


In [1]:
!ls ./fluency_balanced

ls: cannot access './fluency_balanced_7_series': No such file or directory


### Splitting to train, val, test sets

In [5]:
import random
import shutil
import os

# Specify the source directory where your files are located
source_directory = './fluency_balanced'

# Specify the destination directories for the split files
destination_directory_1 = './train_fluency'
destination_directory_2 = './val_fluency'
destination_directory_3 = './test_fluency'

# Create the destination directories if they don't exist
for directory in [destination_directory_1, destination_directory_2, destination_directory_3]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Shuffle the files in the source directory
files = os.listdir(source_directory)
random.shuffle(files)

# Define the sizes for each split
split_sizes = [0.8, 0.1, 0.1]

# Calculate the number of files for each split
total_files = len(files)
split_counts = [int(size * total_files) for size in split_sizes]

# Split the files into different lists based on the split counts
file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

count = 0
# Move the files to the destination directories
for i, file_split in enumerate(file_splits):
    if i == 0:
        destination_directory = destination_directory_1
    elif i == 1:
        destination_directory = destination_directory_2
    else:
        destination_directory = destination_directory_3

    for filename in file_split:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
        count += 1

print(count)

1853


In [6]:
# Specify the directory where your .wav files are located
directory1 = './train_fluency'
directory2 = './val_fluency'
directory3 = './test_fluency'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    if filename.startswith('nostutter'):
        count_train += 1
    if filename.endswith('.wav'):
        count_train_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    if filename.startswith('nostutter'):
        count_val += 1
    if filename.endswith('.wav'):
        count_val_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    if filename.startswith('nostutter'):
        count_test += 1    
    if filename.endswith('.wav'):
        count_test_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

print('**********Sep28 dataset**********')
print(f"Number of nostutter files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of nostutter files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of nostutter files in the test folder: {count_test} and tot is {count_test_tot}")

**********Sep28 dataset**********
Number of nostutter files in the train folder: 736 and tot is 1483
Number of nostutter files in the val folder: 101 and tot is 185
Number of nostutter files in the test folder: 93 and tot is 185


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c74a7a1-d505-4b2b-a4fa-efea0c0f20da' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>